In [1]:
import sys
import pathlib
import pandas as pd
import numpy as np

from sklearn.linear_model import LinearRegression
import plotly.express as px
from pycytominer.cyto_utils import infer_cp_features
from matplotlib import pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder
import pdfkit
import pyarrow as pa
import pyarrow.parquet as pq

# import Union
from typing import Union

sys.path.append("..")
# from ..utils.utils import df_stats
import matplotlib.pyplot as plt

In [2]:
# Define inputs
feature_file = pathlib.Path(
    "../../Extracted_Features_(CSV_files)/feature_df_sc_norm.parquet"
)
feature_df = pq.read_table(feature_file).to_pandas()
# feature_df = pd.read_csv(feature_file, engine="pyarrow")

In [3]:
# define output file path
simple_model_output_file_path = pathlib.Path("./results/lm_one_beta")

complex_model_output_file_path = pathlib.Path("./results/lm_two_beta")

filename = pathlib.Path("figures/lm_one_beta")
filename = pathlib.Path("figures/lm_two_beta")

In [9]:
cp_features = infer_cp_features(feature_df)
print(f"We are testing {len(cp_features)} CellProfiler features")

new_line = "\n"
print(
    f"The unique Treatment-Dosages are: {f', {new_line}'.join((feature_df['Metadata_Treatment_and_Dose'].unique()))}"
)

We are testing 2847 CellProfiler features
The unique Treatment-Dosages are: media ctr_0, 
DMSO_0.100, 
Thapsigargin_1.000, 
Thapsigargin_10.000, 
Topotecan_5.000, 
Topotecan_10.000, 
Topotecan_20.000, 
LPS_0.010, 
LPS_0.100, 
LPS_1.000, 
LPS_10.000, 
LPS_100.000, 
Disulfiram_0.100, 
Disulfiram_1.000, 
Disulfiram_2.500, 
H2O2_100.000, 
Flagellin_0.100, 
Flagellin_1.000


##### Here I plot the beta coefficients for each treatment against the number of cells per well. Data points the drift heavily in the Y axis are features that are affected the most by the y-axis treatment while data points that drift more in the x-axis are features that are most affected by the number of cells in a well.  

#### Simple Linear Modeling (cell count beta + 1 beta approach)
Here I merged the treatment and dosage and used DMSO 0.1% as the control simply comparing one dosage/treatment at a time and outputting each graph for each treatment for all features. All features and treatments will be exported into 1 file.

Linear Model:  
$y = \beta _{0}x+ \beta _{1}x+ \epsilon$ where;  
$y$ is each feature    
$x$ is the inputed variables  
$\beta _{0}$ is the beta coefficient attributed to cell count,  
$\beta _{1}$ is the beta coefficient attributed to treatment & dose combined.  
$\epsilon$ is the residual variance not explained by factors in the model

In [ ]:
feature_df

In [ ]:
# pd.set_option("mode.chained_assignment", None)

model_covariates = ["Metadata_number_of_singlecells"]
control = "DMSO 0.1%_0"
lm_results = []
for treatment in feature_df["Metadata_Treatment_and_Dose"].unique():
    dosage_treatments_list = [treatment, control]
    # filter df for treatment and dose
    df = feature_df.query("Metadata_Treatment_and_Dose in @dosage_treatments_list")
    # encode treatment and dose as integers

    df["Metadata_Treatment_and_Dose"] = LabelEncoder().fit_transform(
        df["Metadata_Treatment_and_Dose"]
    )

    # Setup linear modeling framework

    X = df.loc[:, model_covariates]
    X = pd.concat([X, df["Metadata_Treatment_and_Dose"]], axis=1)

    columns_list = (
        ["feature", "r2_score"] + X.columns.tolist() + ["dosage_treatments_list"]
    )

    # Fit linear model for each feature
    for cp_feature in cp_features:
        # Subset CP data to each individual feature (univariate test)
        cp_subset_df = df.loc[:, cp_feature]

        # Fit linear model
        lm = LinearRegression(fit_intercept=True)
        lm_result = lm.fit(X=X, y=cp_subset_df)

        # Extract Beta coefficients(contribution of feature to X covariates)
        coef = list(lm_result.coef_)
        # Estimate fit (R^2)
        r2_score = lm.score(X=X, y=cp_subset_df)

        # Add results to a growing list
        lm_results.append(
            [cp_feature, r2_score] + coef + [f"{'-'.join(dosage_treatments_list)}"]
        )

# Convert results to a pandas DataFrame
lm_results_df = pd.DataFrame(lm_results, columns=columns_list)

simple_model_output_file_path = (
    f'{simple_model_output_file_path}_{"_".join(model_covariates)}.tsv'
)

# write output to file
lm_results_df.to_csv(simple_model_output_file_path, sep="\t", index=False)

#### Complex Linear Modeling (cell count btea + 2 beta approach)
Here I run the same analysis as above but with dosage of a treatment being a factor in the linear model. All features and treatments will be exported into 1 file.

Linear Model:  
$y = \beta _{0}x+ \beta _{1}x+ \beta _{2}x+ \epsilon$ where;  
$y$ is each feature    
$x$ is the inputed variables  
$\beta _{0}$ is the beta coefficient attributed to cell count.  
$\beta _{1}$ is the beta coefficient attributed to treatment.  
$\beta _{2}$ is the beta coefficient attributed to dose.  
$\epsilon$ is the residual variance not explained by factors in the model

In [4]:
feature_df

,Metadata_cell_type,Metadata_Well,Metadata_number_of_singlecells,Metadata_incubation inducer (h),Metadata_inhibitor,Metadata_inhibitor_concentration,Metadata_inhibitor_concentration_unit,Metadata_inducer1,Metadata_inducer1_concentration,Metadata_inducer1_concentration_unit,...,Nuclei_Texture_Variance_CorrGasdermin_3_03_256,Nuclei_Texture_Variance_CorrMito_3_00_256,Nuclei_Texture_Variance_CorrMito_3_01_256,Nuclei_Texture_Variance_CorrMito_3_02_256,Nuclei_Texture_Variance_CorrMito_3_03_256,Nuclei_Texture_Variance_CorrPM_3_00_256,Nuclei_Texture_Variance_CorrPM_3_01_256,Nuclei_Texture_Variance_CorrPM_3_02_256,Nuclei_Texture_Variance_CorrPM_3_03_256,Metadata_Treatment_and_Dose
0,SH-SY5Y,B13,3765,6,Media ctr,0.0,None,media ctr,0.0,None,...,0.003817,0.353469,0.365618,0.378125,0.342100,0.099225,0.096054,0.110432,0.107764,media ctr_0
1,SH-SY5Y,B13,3765,6,Media ctr,0.0,None,media ctr,0.0,None,...,-0.042322,1.780397,1.603914,1.571446,1.582787,0.020683,0.016122,0.012990,0.013631,media ctr_0
2,SH-SY5Y,B13,3765,6,Media ctr,0.0,None,media ctr,0.0,None,...,-0.054760,0.019005,0.041005,0.029673,0.007339,0.027519,0.045982,0.037653,0.017281,media ctr_0
3,SH-SY5Y,B13,3765,6,Media ctr,0.0,None,media ctr,0.0,None,...,-0.059141,-0.005212,-0.010673,0.000258,0.003475,-0.002945,0.000293,0.008048,-0.002151,media ctr_0
4,SH-SY5Y,B13,3765,6,Media ctr,0.0,None,media ctr,0.0,None,...,-0.030255,-0.044078,-0.052547,-0.042483,-0.029738,0.016846,0.006499,0.014114,0.026206,media ctr_0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
597897,SH-SY5Y,O23,3555,6,Media ctr,0.0,None,media ctr,0.0,None,...,-0.068560,-0.117209,-0.125985,-0.127684,-0.122355,-0.024370,-0.033577,-0.033811,-0.028148,media ctr_0
597898,SH-SY5Y,O23,3555,6,Media ctr,0.0,None,media ctr,0.0,None,...,-0.007537,-0.108656,-0.110885,-0.112064,-0.113660,-0.049855,-0.048752,-0.050176,-0.049417,media ctr_0
597899,SH-SY5Y,O23,3555,6,Media ctr,0.0,None,media ctr,0.0,None,...,-0.050434,-0.088444,-0.089816,-0.092630,-0.092441,-0.027676,-0.024066,-0.026279,-0.027050,media ctr_0
597900,SH-SY5Y,O23,3555,6,Media ctr,0.0,None,media ctr,0.0,None,...,0.081602,0.098873,0.110850,0.109012,0.112935,0.308702,0.309290,0.311365,0.309627,media ctr_0


In [12]:
# Loop for each treatment then each feature

# define the control and treatment
# Setup linear modeling framework
model_covariates = ["Metadata_number_of_singlecells"]
control = "DMSO"
lm_results = []
for treatment in feature_df["Metadata_inducer1"].unique():
    dosage_treatments_list = [treatment, control]
    print(dosage_treatments_list)
    df = feature_df.query("Metadata_inducer1 in @dosage_treatments_list")
    # Add dummy matrix of categorical genotypes
    # treatment_df = feature_df[["Metadata_inducer1", "Metadata_inducer1_concentration"]]
    tmp_df = df.loc[:, ("Metadata_inducer1", "Metadata_inducer1_concentration")]
    tmp_df["Metadata_inducer1"] = LabelEncoder().fit_transform(
        tmp_df["Metadata_inducer1"]
    )
    tmp_df["Metadata_inducer1_concentration"] = LabelEncoder().fit_transform(
        tmp_df["Metadata_inducer1_concentration"]
    )

    X = pd.concat([df.loc[:, model_covariates], tmp_df], axis=1)
    columns_list = ["feature", "r2_score"] + X.columns.tolist() + ["treatment", "dose"]

    # Fit linear model for each feature
    # lm_results = []
    for cp_feature in cp_features:
        # Subset CP data to each individual feature (univariate test)
        cp_subset_df = df.loc[:, cp_feature]

        # Fit linear model
        lm = LinearRegression(fit_intercept=True)
        lm_result = lm.fit(X=X, y=cp_subset_df)

        # Extract Beta coefficients
        # (contribution of feature to X covariates)
        coef = list(lm_result.coef_)
        # Estimate fit (R^2)
        r2_score = lm.score(X=X, y=cp_subset_df)

        # Add results to a growing list
        lm_results.append(
            [cp_feature, r2_score]
            + coef
            + [
                treatment,
                "_".join(df["Metadata_inducer1_concentration"].astype(str).unique()),
            ]
        )

# Convert results to a pandas DataFrame
lm_results_df = pd.DataFrame(lm_results, columns=columns_list)

# define output file path
complex_model_output_file_path = (
    f'{complex_model_output_file_path}_{"_".join(model_covariates)}.tsv'
)

# write output to file
lm_results_df.to_csv(complex_model_output_file_path, sep="\t", index=False)

['media ctr', 'DMSO']
['DMSO', 'DMSO']
['Thapsigargin', 'DMSO']
['Topotecan', 'DMSO']
['LPS', 'DMSO']
['Disulfiram', 'DMSO']
['H2O2', 'DMSO']
['Flagellin', 'DMSO']


In [ ]:
feature_df["Metadata_inducer1"].unique()